# EXTRACT SKILLS 

## Xilin Tian

In [8]:
import jsonlines
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
resume = pd.read_csv ('resume_add_skills.csv', encoding = 'ISO-8859-1', usecols = [0,1,2])
resume = resume.rename(columns = {'ï»¿ID':'ID'})
resume

,ID,Category,Resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."
...,...,...,...
3672,99416532,AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...
3673,24589765,AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ..."
3674,31605080,AVIATION,GEEK SQUAD AGENT Professional...
3675,21190805,AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...


In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
from spacy.pipeline import EntityRuler
ruler = nlp.add_pipe("entity_ruler", name="ruler", before="ner")
ruler.from_disk("jz_skill_patterns.jsonl")
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ruler',
 'ner']

In [5]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [23]:
def clean(text):
    delete_n = re.sub(r'\\n', ' ', text)
    delete_x = re.sub(r'\\x[a-z0-9]+', ' ', delete_n)
    return delete_x

In [25]:
clean_text = []
for i in range(3677):
    resume_clean = clean(resume['Resume'].loc[i])
    clean_text.append(resume_clean)

In [26]:
resume_skills = []
for i in range(3677):
    resume_extract = unique_skills(get_skills(resume['Resume'].loc[i]))
    resume_skills.append(resume_extract)

In [28]:
resume['skills'] = resume_skills
resume

,ID,Category,Resume,skills
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...","[Collaboration, security, Box, Data Management..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,"[databases, business]"
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,"[support, first class, Business, communication..."
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,"[Testing, database, Multimedia, Java, Mobile, ..."
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...","[support, collaboration, Engineering, Communic..."
...,...,...,...,...
3672,99416532,AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"[Engineering, material, Accounting]"
3673,24589765,AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","[design, support, Support, finance, communicat..."
3674,31605080,AVIATION,GEEK SQUAD AGENT Professional...,"[IOS, support, Software, Android, Windows, sof..."
3675,21190805,AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...,"[Twitter, support, design, Operating Systems, ..."


In [32]:
resume.to_csv('Extract skills.csv', index=False)